# Weather and Motor Vehicle Collisions

In [4]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from dateutil.rrule import rrule, DAILY
from __future__ import division
import geoplotlib as glp
from geoplotlib.utils import BoundingBox, DataAccessObject

pd.set_option('display.max_columns', None)
%matplotlib inline  

## Read weather data from csv file

In [89]:
weather = pd.read_csv('datasets/weather_data_nyc_clean2.csv')
incidents = pd.read_csv("datasets/NYPD_Motor_Vehicle_Collisions_weather4.csv")

In [91]:
weather['date'] = weather.Year.astype('str') +'/'+ weather.Month.astype('str') +'/'+ weather.Day.astype('str') +'/'+ weather.Hour.astype('str')


In [92]:
weather.Conditions.unique()

array(['Clear', 'Partly Cloudy', 'Scattered Clouds', 'Unknown',
       'Mostly Cloudy', 'Overcast', 'Light Rain', 'Heavy Rain', 'Rain',
       'Haze', 'Fog', 'Mist', 'Light Snow', 'Heavy Snow', 'Snow',
       'Light Freezing Rain', 'Light Freezing Fog'], dtype=object)

*** Frequency of measured weather conditions from 7/1/2012 to 3/1/2016, on hourly basis ***

In [93]:
weatherConditionCount = {'Clear':0, 'Partly Cloudy':0, 'Scattered Clouds':0, 'Unknown':0,
       'Mostly Cloudy':0, 'Overcast':0, 'Light Rain':0, 'Heavy Rain':0, 'Rain':0,
       'Haze':0, 'Fog':0, 'Mist':0, 'Light Snow':0, 'Heavy Snow':0, 'Snow':0,
       'Light Freezing Rain':0, 'Light Freezing Fog':0}

for d in weather.date.unique():
    condi = weather[weather.date == d]['Conditions'].iloc[0]
    weatherConditionCount[condi] += 1

weatherConditionCount

{'Clear': 16042,
 'Fog': 34,
 'Haze': 742,
 'Heavy Rain': 132,
 'Heavy Snow': 25,
 'Light Freezing Fog': 1,
 'Light Freezing Rain': 18,
 'Light Rain': 1149,
 'Light Snow': 332,
 'Mist': 6,
 'Mostly Cloudy': 2504,
 'Overcast': 7169,
 'Partly Cloudy': 1665,
 'Rain': 290,
 'Scattered Clouds': 1401,
 'Snow': 88,
 'Unknown': 410}

## Get frequency of collision per hour of specific weather condition

In [85]:
conditionCount = {}
for c in incidents.Conditions.unique():
    if (pd.notnull(c)):
        mask = ((incidents.Conditions == c))
        filtered_incidents = incidents[mask]
        conditionCount[c] = filtered_incidents.size
    
conditionCount


    
    

{'Blowing Snow': 884,
 'Clear': 1043732,
 'Fog': 116518,
 'Haze': 112200,
 'Heavy Rain': 82246,
 'Heavy Snow': 5644,
 'Heavy Thunderstorms and Rain': 21522,
 'Ice Pellets': 1496,
 'Light Drizzle': 229636,
 'Light Freezing Drizzle': 4760,
 'Light Freezing Rain': 30770,
 'Light Ice Pellets': 13090,
 'Light Rain': 1548632,
 'Light Rain Showers': 1326,
 'Light Snow': 479026,
 'Light Thunderstorms and Rain': 52258,
 'Mist': 9826,
 'Mostly Cloudy': 8756700,
 'Overcast': 3677100,
 'Partly Cloudy': 2912236,
 'Patches of Fog': 2346,
 'Rain': 267410,
 'Scattered Clouds': 5272788,
 'Shallow Fog': 3366,
 'Snow': 45628,
 'Squalls': 1564,
 'Thunderstorm': 29104,
 'Thunderstorms and Rain': 8670,
 'Thunderstorms with Small Hail': 1190,
 'Unknown': 6834}

In [86]:
for k,v in conditionCount.iteritems():
    conditionCountValue = conditionCount[k]
    weatherConditionCountValue = weatherConditionCount[k]
    print v

KeyError: 'Light Drizzle'